<p style="font-size: 45px; text-align: center;"><b>Machine Learning in Finance II - final project</b></p>
<p style="font-size: 35px; text-align: center;"><b>Forecasting delays in delivery time - Brazilian E-commerce</b></p>
<p style="font-size: 20px; text-align: center;"><b>Neural Networks</b></p>

Author: Jakub Pyszniak

Notebook 6

> **Post deadline experimentation with neural networks (practice exercises)**

# For future consideration

> **Consider cleaning up the code and building a framework to collect the results of different NN set-ups**

> **Add appropriate comments**

# Libraries

In [55]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SelectFromModel
from scipy import stats
import math
import time
from scipy.stats import randint, uniform, loguniform
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error

# Visual set-up
pd.set_option("display.max_columns", 60)


# NN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# These data will be used for CV and choosing the best models
df_train = pd.read_csv("4.train_and_test/df_train.csv")

# We will use this set to test the forecasting power of our best models (final comparisons)
df_test = pd.read_csv("4.train_and_test/df_test.csv")

# Our feature ranking dataset
fr = pd.read_excel("3.feature_ranking/feature_ranking.xlsx")

In [4]:
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73180 entries, 0 to 73179
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   delay_days                     73180 non-null  float64
 1   order_item_id                  73180 non-null  int64  
 2   total_price                    73180 non-null  float64
 3   total_freight                  73180 non-null  float64
 4   order_value_check              73180 non-null  float64
 5   is_payment_sequential_1        73180 non-null  int64  
 6   payment_type_boleto            73180 non-null  int64  
 7   payment_type_credit_card       73180 non-null  int64  
 8   payment_type_debit_card        73180 non-null  int64  
 9   payment_type_voucher           73180 non-null  int64  
 10  installments_boleto            73180 non-null  int64  
 11  installments_credit_card       73180 non-null  int64  
 12  installments_debit_card        73180 non-null 

,delay_days,order_item_id,total_price,total_freight,order_value_check,is_payment_sequential_1,payment_type_boleto,payment_type_credit_card,payment_type_debit_card,payment_type_voucher,installments_boleto,installments_credit_card,installments_debit_card,installments_voucher,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,seller_state,customer_state,cust_sell_same_state,customer_lat,customer_lng,seller_lat,seller_lng,cust_sell_distance_km,order_purchase_month,order_delivery_month,order_purchase_dow,order_delivery_dow,order_estimate_dow
0,-21.0,1,19.99,14.10,34.09,1,0,1,0,0,0,1,0,0,48.0,575.0,1,100.0,20.0,15.0,15.0,43,21,18,0,-22.761992,-43.450873,-23.665703,-46.518082,329.149657,6,6,5,0,0
1,-4.0,1,72.90,19.70,92.60,1,0,1,0,0,0,1,0,0,37.0,360.0,1,650.0,45.0,15.0,25.0,67,14,25,0,-21.679558,-49.762053,-23.179392,-50.634922,189.372236,8,8,0,3,0
2,-38.0,1,50.90,15.57,66.47,1,0,1,0,0,0,1,0,0,60.0,473.0,1,600.0,30.0,4.0,20.0,39,21,18,0,-22.449744,-43.474330,-21.766477,-48.831547,557.068979,2,2,3,4,0
3,-8.0,1,199.90,14.23,214.13,1,0,1,0,0,0,5,0,0,60.0,233.0,1,2600.0,41.0,8.0,36.0,7,21,25,1,-23.640572,-46.570773,-22.716839,-47.657366,151.282904,6,6,0,2,3
4,-12.0,2,44.00,35.26,79.26,1,0,1,0,0,0,7,0,0,58.0,1623.0,1,200.0,26.0,10.0,22.0,7,5,18,0,-22.983577,-43.220723,-16.692331,-49.268016,942.794225,3,3,4,2,0


In [5]:
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18295 entries, 0 to 18294
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   delay_days                     18295 non-null  float64
 1   order_item_id                  18295 non-null  int64  
 2   total_price                    18295 non-null  float64
 3   total_freight                  18295 non-null  float64
 4   order_value_check              18295 non-null  float64
 5   is_payment_sequential_1        18295 non-null  int64  
 6   payment_type_boleto            18295 non-null  int64  
 7   payment_type_credit_card       18295 non-null  int64  
 8   payment_type_debit_card        18295 non-null  int64  
 9   payment_type_voucher           18295 non-null  int64  
 10  installments_boleto            18295 non-null  int64  
 11  installments_credit_card       18295 non-null  int64  
 12  installments_debit_card        18295 non-null 

,delay_days,order_item_id,total_price,total_freight,order_value_check,is_payment_sequential_1,payment_type_boleto,payment_type_credit_card,payment_type_debit_card,payment_type_voucher,installments_boleto,installments_credit_card,installments_debit_card,installments_voucher,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,seller_state,customer_state,cust_sell_same_state,customer_lat,customer_lng,seller_lat,seller_lng,cust_sell_distance_km,order_purchase_month,order_delivery_month,order_purchase_dow,order_delivery_dow,order_estimate_dow
0,-15.0,1,108.0,16.52,124.52,1,0,1,0,0,0,1,0,0,58.0,3006.0,2,1000.0,53.0,8.0,18.0,72,15,10,0,-21.248980,-44.998179,-22.874599,-43.477731,239.212282,2,2,5,3,4
1,-2.0,1,78.0,7.80,85.80,1,0,1,0,0,0,2,0,0,59.0,319.0,4,250.0,16.0,2.0,20.0,72,21,25,1,-23.657047,-46.774874,-23.651115,-46.755211,2.108617,11,11,4,1,3
2,-19.0,1,199.9,15.15,215.05,1,0,1,0,0,0,5,0,0,55.0,623.0,1,337.0,16.0,13.0,13.0,72,21,18,0,-22.704280,-43.571287,-22.828655,-47.267296,379.200398,6,6,3,2,0
3,-17.0,1,69.0,19.45,88.45,1,0,1,0,0,0,6,0,0,51.0,324.0,1,900.0,42.0,8.0,37.0,7,21,18,0,-22.747569,-43.488349,-21.766477,-48.831547,560.549267,6,6,3,0,3
4,-5.0,1,44.9,9.42,54.32,1,1,0,0,0,1,0,0,0,51.0,1118.0,5,400.0,18.0,8.0,14.0,49,21,25,1,-23.663579,-46.617176,-23.198860,-47.293346,86.193580,3,3,3,3,1


In [6]:
fr.rename(columns={"Unnamed: 0": "feature"}, inplace=True)

fr = fr.sort_values(by=["boruta_rank","mi_score"], ascending=[True, False])

fr.info()
fr.head(40)

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 2 to 11
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   feature      34 non-null     object 
 1   mi_score     34 non-null     float64
 2   boruta_rank  34 non-null     int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ KB


,feature,mi_score,boruta_rank
2,total_freight,0.071483,1
29,order_purchase_month,0.067143,1
30,order_delivery_month,0.063950,1
27,seller_lng,0.057679,1
28,cust_sell_distance_km,0.052569,1
24,customer_lat,0.046624,1
25,customer_lng,0.044898,1
23,cust_sell_same_state,0.043066,1
21,seller_state,0.020188,1
13,product_name_length,0.003299,1


In [7]:
fr = fr.set_index("feature")

In [8]:
br10_feat = fr.iloc[0:10].index.tolist()
print("10 best:\n", br10_feat)

br15_feat = fr.iloc[0:15].index.tolist()
print("15 best:\n", br15_feat)

br20_feat = fr.iloc[0:20].index.tolist()
print("20 best:\n", br20_feat)

br25_feat = fr.iloc[0:25].index.tolist()
print("25 best:\n", br25_feat)

br30_feat = fr.iloc[0:30].index.tolist()
print("30 best:\n", br30_feat)

10 best:
 ['total_freight', 'order_purchase_month', 'order_delivery_month', 'seller_lng', 'cust_sell_distance_km', 'customer_lat', 'customer_lng', 'cust_sell_same_state', 'seller_state', 'product_name_length']
15 best:
 ['total_freight', 'order_purchase_month', 'order_delivery_month', 'seller_lng', 'cust_sell_distance_km', 'customer_lat', 'customer_lng', 'cust_sell_same_state', 'seller_state', 'product_name_length', 'product_width_cm', 'order_estimate_dow', 'product_weight_g', 'seller_lat', 'product_height_cm']
20 best:
 ['total_freight', 'order_purchase_month', 'order_delivery_month', 'seller_lng', 'cust_sell_distance_km', 'customer_lat', 'customer_lng', 'cust_sell_same_state', 'seller_state', 'product_name_length', 'product_width_cm', 'order_estimate_dow', 'product_weight_g', 'seller_lat', 'product_height_cm', 'order_value_check', 'customer_state', 'product_description_length', 'total_price', 'order_delivery_dow']
25 best:
 ['total_freight', 'order_purchase_month', 'order_delivery_mo

In [ ]:
# build X / y splits using only the selected features

X_train  = df_train.loc[:, df_train.columns !="delay_days"].copy()
X10_train = df_train.loc[:, br10_feat].copy()
X15_train = df_train.loc[:, br15_feat].copy()
X20_train = df_train.loc[:, br20_feat].copy()
X25_train = df_train.loc[:, br25_feat].copy()
X30_train = df_train.loc[:, br30_feat].copy()
y_train = df_train.loc[:, "delay_days"].copy()

X_test  = df_test.loc[:, df_test.columns !="delay_days"].copy()
X10_test  = df_test.loc[:, br10_feat].copy()
X15_test  = df_test.loc[:, br15_feat].copy()
X20_test  = df_test.loc[:, br20_feat].copy()
X25_test  = df_test.loc[:, br25_feat].copy()
X30_test  = df_test.loc[:, br30_feat].copy()
y_test  = df_test.loc[:, "delay_days"].copy()

# Let's start with just 10 features

In [38]:
X_test.nunique()

order_item_id                        8
total_price                       2952
total_freight                     3840
order_value_check                10651
is_payment_sequential_1              2
payment_type_boleto                  2
payment_type_credit_card             2
payment_type_debit_card              2
payment_type_voucher                 2
installments_boleto                  2
installments_credit_card            19
installments_debit_card              2
installments_voucher                 2
product_name_length                 59
product_description_length        2224
product_photos_qty                  18
product_weight_g                  1271
product_length_cm                   96
product_height_cm                   97
product_width_cm                    85
product_category_name_english       70
seller_state                        20
customer_state                      27
cust_sell_same_state                 2
customer_lat                      8344
customer_lng             

In [ ]:
# ============================================================
# 1) Define which columns are categorical vs numeric
# ============================================================
# These are your categorical columns (from your boosting setup).
# We will NOT one-hot encode them. Instead we:
#   - map each category to an integer id
#   - feed those ids into an Embedding layer

cat_cols = [
    "order_purchase_month", "order_delivery_month", "seller_state" , "order_estimate_dow",
    "customer_state", "order_delivery_dow", "product_category_name_english", "order_purchase_dow",
    "order_delivery_dow", "order_estimate_dow"
]


##--- One variable to switch between feature sets
FEATURE_SET = 10  # <- change this to 34, 25, 20, 15

X_sets = {
    10: (X10_train, X10_test),
    15: (X15_train, X15_test),
    20: (X20_train, X20_test),
    25: (X25_train, X25_test),
    30: (X30_train, X30_test),
    34: (X_train, X_test),
}

Xn_train, Xn_test = X_sets[FEATURE_SET]
##---

# keep only the ones that actually exist in X10_train
cat_cols = [c for c in cat_cols if c in Xn_train.columns]

# Everything else (remaining columns) we treat as numeric.
# Numeric features should be standardized for neural nets.
num_cols = [c for c in Xn_train.columns if c not in cat_cols]

In [44]:
# ============================================================
# 2) Copy train/test to avoid altering original data
# ============================================================
# It's good practice not to mutate your original dataframes.
Xtr_n = Xn_train.copy()
Xte_n = Xn_test.copy()

# Make sure y is a NumPy array of shape (n, 1) for Keras regression.
ytr = np.asarray(y_train).reshape(-1, 1).astype("float32")
yte = np.asarray(y_test).reshape(-1, 1).astype("float32")

In [ ]:
# ============================================================
# 3) Scale numeric features
# ============================================================
# Neural nets are sensitive to feature scale.
# If one numeric feature ranges 0..100000 and another is 0..1,
# the big one dominates gradients and training becomes unstable.

# StandardScaler transforms each numeric column to:
#   (x - mean) / std
scaler_x = StandardScaler()

# Fit scaler ONLY on training numeric features (avoid leakage!)
Xtr_num = scaler_x.fit_transform(Xtr_n[num_cols])

# Use the SAME scaler for the test set
Xte_num = scaler_x.transform(Xte_n[num_cols])

In [ ]:

# ============================================================
# 4) Encode categorical features into integer ids
# ============================================================
# Embedding layers require integer indices (0, 1, 2, ...).
#
# We create a per-column vocabulary from TRAIN ONLY.
# Anything unseen in test will be mapped to an "unknown" id.
#
# Example:
#   seller_state vocab = ["SP","RJ","MG",...]
#   "SP" -> 0, "RJ" -> 1, "MG" -> 2, ...
#   unseen in test -> unk_id (last index)
cat_maps = {}   # will store vocabularies (categories) for each column
Xtr_cat = {}    # will store integer arrays for train
Xte_cat = {}    # will store integer arrays for test

for col in cat_cols:
    # Convert to string and fill missing values:
    # - Embeddings don't handle NaN directly.
    # - We turn missing into a literal category "NA".
    tr_vals = Xtr_n[col].astype(str).fillna("NA")
    te_vals = Xte_n[col].astype(str).fillna("NA")

    # Vocabulary: unique categories in TRAIN ONLY
    vocab = pd.Index(tr_vals.unique())
    cat_maps[col] = vocab  # save it for later (and for inference)

    # get_indexer maps each value to its index in vocab
    # if something is not found, it returns -1
    tr_ids = vocab.get_indexer(tr_vals)
    te_ids = vocab.get_indexer(te_vals)

    # Unknown category id (one beyond last known id)
    unk_id = len(vocab)

    # Replace unseen test categories (-1) with unk_id
    te_ids = np.where(te_ids == -1, unk_id, te_ids)

    # Store as int32 (Keras Embedding expects int32/int64 indices)
    Xtr_cat[col] = tr_ids.astype("int32")
    Xte_cat[col] = te_ids.astype("int32")

In [47]:
# ============================================================
# 5) Build the Keras model: numeric input + embedding inputs
# ============================================================
# We create:
# - one Input() for the numeric matrix
# - one Input() per categorical column (shape=(1,))
# - each categorical input goes through an Embedding layer
# - concatenate everything and pass through Dense layers (MLP)
#
# Output is one scalar prediction: regression.
num_in = keras.Input(shape=(Xtr_num.shape[1],), name="num")

cat_inputs = []  # list of Keras inputs for categorical columns
cat_embeds = []  # list of embedding outputs

for col in cat_cols:
    # vocab_size: number of training categories + 1 for unknown bucket
    vocab_size = len(cat_maps[col]) + 1

    # Embedding dimension heuristic:
    # - Small vocab -> small embedding
    # - Large vocab -> bigger embedding (capped)
    # The formula below is a simple "rule of thumb".
    emb_dim = int(min(50, round((vocab_size ** 0.25) * 4)))

    # Each categorical column is a single integer per row => shape=(1,)
    inp = keras.Input(shape=(1,), dtype="int32", name=col)

    # Embedding layer turns an integer id into a dense vector (length emb_dim)
    emb = layers.Embedding(
        input_dim=vocab_size,
        output_dim=emb_dim,
        name=f"emb_{col}"
    )(inp)

    # Embedding output shape is (batch, 1, emb_dim)
    # We reshape to (batch, emb_dim) so we can concatenate later
    emb = layers.Reshape((emb_dim,))(emb)

    cat_inputs.append(inp)
    cat_embeds.append(emb)

# Concatenate numeric + all embeddings into one big feature vector
x = layers.Concatenate(name="concat_features")([num_in] + cat_embeds)

In [48]:
# ============================================================
# 6) MLP "head" (dense layers)
# ============================================================
# This is the actual neural network doing the regression.
# - Dense layers learn nonlinear combinations of features.
# - Dropout helps reduce overfitting (randomly zeroes activations during training).
x = layers.Dense(256, activation="relu", name="dense_256")(x)
x = layers.Dropout(0.2, name="dropout_0p2")(x)

x = layers.Dense(128, activation="relu", name="dense_128")(x)
x = layers.Dropout(0.1, name="dropout_0p1")(x)

x = layers.Dense(64, activation="relu", name="dense_64")(x)

# Final output: single scalar regression value
out = layers.Dense(1, name="y")(x)

# Build model object
model = keras.Model(inputs=[num_in] + cat_inputs, outputs=out, name="tabular_embedding_mlp")

In [49]:
# ============================================================
# 7) Compile the model (optimizer, loss, metrics)
# ============================================================
# Loss:
# - MSE is standard for regression
# Metric:
# - RMSE is easier to interpret (same units as target)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="mse",
    metrics=[keras.metrics.RootMeanSquaredError(name="rmse")]
)

# Print model architecture
model.summary()

Model: "tabular_embedding_mlp"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ order_purchase_mon… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ order_delivery_mon… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ seller_state        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_order_purchase… │ (None, 1, 8)      │        104 │ order_purchase_m… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_order_delivery… │ (None, 1, 8)      │        104 │ order_delivery_m… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_seller_state    │ (None, 1, 9)      │        207 │ seller_state[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ num (InputLayer)    │ (None, 7)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 8)         │          0 │ emb_order_purcha… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 8)         │          0 │ emb_order_delive… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 9)         │          0 │ emb_seller_state… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat_features     │ (None, 32)        │          0 │ num[0][0],        │
│ (Concatenate)       │                   │            │ reshape[0][0],    │
│                     │                   │            │ reshape_1[0][0],  │
│                     │                   │            │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_256 (Dense)   │ (None, 256)       │      8,448 │ concat_features[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_0p2         │ (None, 256)       │          0 │ dense_256[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_128 (Dense)   │ (None, 128)       │     32,896 │ dropout_0p2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_0p1         │ (None, 128)       │          0 │ dense_128[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_64 (Dense)    │ (None, 64)        │      8,256 │ dropout_0p1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ y (Dense)           │ (None, 1)         │         65 │ dense_64[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 50,080 (195.62 KB)

 Trainable params: 50,080 (195.62 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# ============================================================
# 8) Prepare inputs in the format Keras expects
# ============================================================
# Keras will match dict keys to Input layer names.
# We create a dictionary like:
#   {"num": X_numeric_matrix,
#    "seller_state": int_ids,
#    "customer_state": int_ids, ...}
train_inputs = {"num": Xtr_num}
test_inputs  = {"num": Xte_num}

for col in cat_cols:
    train_inputs[col] = Xtr_cat[col]
    test_inputs[col]  = Xte_cat[col]

In [51]:
# ============================================================
# 9) Callbacks for stable training
# ============================================================
# EarlyStopping:
# - if validation RMSE stops improving for patience epochs, stop training
# - restore best weights so we keep the best-performing epoch
#
# ReduceLROnPlateau:
# - if learning stalls, reduce learning rate to fine-tune further
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_rmse",
        patience=10,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_rmse",
        patience=5,
        factor=0.5,
        min_lr=1e-5
    )
]

In [52]:
# ============================================================
# 10) Train the model
# ============================================================
# validation_split=0.1 means:
# - Keras will take 10% of the training data as validation
# - it will NOT touch X_test during training
#
# batch_size:
# - bigger batches often work well for tabular
# - 1024 is a reasonable starting point for ~90k rows
history = model.fit(
    train_inputs, ytr,
    validation_split=0.1,
    epochs=200,
    batch_size=1024,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 129.8064 - rmse: 11.3933 - val_loss: 92.5869 - val_rmse: 9.6222 - learning_rate: 0.0010
Epoch 2/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 94.1507 - rmse: 9.7031 - val_loss: 90.3333 - val_rmse: 9.5044 - learning_rate: 0.0010
Epoch 3/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 92.1830 - rmse: 9.6012 - val_loss: 89.0299 - val_rmse: 9.4356 - learning_rate: 0.0010
Epoch 4/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 90.6775 - rmse: 9.5225 - val_loss: 87.5411 - val_rmse: 9.3563 - learning_rate: 0.0010
Epoch 5/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 88.6339 - rmse: 9.4146 - val_loss: 85.1855 - val_rmse: 9.2296 - learning_rate: 0.0010
Epoch 6/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 86.2727 - rmse: 9.2883 - val_loss: 82.6799 - val_rmse: 9.0928 - learning_rate: 0.0010
Epoch 7/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 83.7643 - rmse: 9.1523 - val_loss: 79.9205 - val_rmse: 8.9398 - l

In [57]:
# ============================================================
# 11) Evaluate + predict on test
# ============================================================
# model.evaluate returns [loss, rmse] because we compiled with metric rmse
eval_res = model.evaluate(test_inputs, yte, verbose=0)
print("Test evaluation:", dict(zip(model.metrics_names, eval_res)))

# Predict test set
pred_nn = model.predict(test_inputs, verbose=0).ravel()

# Compute RMSE using sklearn as well (double-check)
rmse_nn = root_mean_squared_error(np.asarray(y_test).ravel(), pred_nn)
print(f"Keras NN Test RMSE (sklearn): {rmse_nn:.4f}")


Test evaluation: {'loss': 59.545894622802734, 'compile_metrics': 7.7165985107421875}
Keras NN Test RMSE (sklearn): 7.7166


In [58]:
# ============================================================
# 12) Create your comparison table (y_true vs NN_pred)
# ============================================================
df_compare = pd.DataFrame({
    "y_true": np.asarray(y_test).ravel(),
    "NN_pred": pred_nn
})

print(df_compare.head(20))

    y_true    NN_pred
0    -15.0 -14.837687
1     -2.0  -8.483032
2    -19.0 -21.661699
3    -17.0 -22.572496
4     -5.0  -8.321438
5     -3.0  -3.879128
6    -13.0 -11.847809
7    -19.0 -19.306744
8    -10.0 -14.152117
9    -20.0 -23.953321
10   -20.0 -16.407118
11    -7.0  -9.280128
12   -14.0 -13.793201
13    -8.0 -10.924683
14    -7.0  -9.150761
15   -19.0 -20.331820
16   -20.0 -19.554499
17    -5.0 -13.040726
18   -10.0  -9.528877
19   -20.0 -19.165087
